## 작업형 제 2 유형

### ✏️ 제품 배송 시간에 맞춰 배송되지 않을 확률값을 예측하시오.
- 제공된 데이터 목록 : X_train, y_train, X_test.csv

- 예측할 컬럼 : Reached.on.Time_Y.N(0: 정시 도착, 1: 정시 도착하지 않음)

- 학습용 데이터(train)을 이용하여 배송 예측 모형을 만든 후 이를 평가용 데이터(test)에 적용하여 얻은 예측 시간(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 csv 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

    - 제출 csv 파일명 : result.csv

    - 컬럼 : ID, Reached.on.Time_Y.N

In [1]:
# 1. 문제 정의
## 배송 관련 데이터. 정시 도착 여부(Reached.on.Time_Y.N) 예측
## 평가 기준은 ROC-AUC 로 평가
## label(target)은 정시 배송 여부(Reached.on.Time_Y.N) -> 0 : 정시 도착, 1 : 정시 도착하지 않음
## 제출 방식은 test 데이터로 정시에 도착할 확률을 csv 파일로 제출 (파일명 : result.csv)

# 2. 라이브러리 및 데이터 불러오기
## 판다스 라이브러리와 주어진 train 과 test 데이터셋 불러오기
## 시험에서 2개 파일이 제공된다면 train 과 test, 3개 파일이 제공된다면 train 데이터에 있는 label(target)을 y_train 으로 별도 제공
import pandas as pd
X_test = pd.read_csv('./data/X_test.csv')
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

---
#### 💡 3개의 데이터가 제공되었을 때 2개가 제공된 것처럼 합치는 방법
- pd.concat() 함수를 활용해 y_train 에 있는 label 값을 합칠 수 있음

In [2]:
import pandas as pd
X_test = pd.read_csv('./data/X_test.csv')
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

df_cc = pd.concat([X_train, y_train['Reached.on.Time_Y.N']], axis=1)
print(df_cc.head())

      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   8440               B             Ship                    4   
1  10849               D             Ship                    3   
2   9478               B             Ship                    4   
3   2968               B             Ship                    6   
4   6964               B             Road                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  284                4                low   
1                2                  201                4             medium   
2                4                  181                3             medium   
3                2                  236                4                low   
4                1                  282                4                low   

  Gender  Discount_offered  Weight_in_gms  Reached.on.Time_Y.N  
0      F                 4           4795                    1 

---

In [3]:
# 3. 탐색적 데이터 분석(EDA)
## 데이터 크기 확인 : y_train 은 X_train 과 행 수는 같고, 열 수는 2개인 것 확인
print(X_train.shape, y_train.shape)

## X_train 데이터 샘플 확인 : 카테고리(문자)와 연속형(숫자) 컬럼 혼합
## y_train 데이터 샘플 확인 : ID 와 label(target) 컬럼 존재, ID 의 경우 X_train 데이터와 순서 동일
print(X_train.head(), '\n===========================================\n', y_train.head(), '\n')

## 데이터 자료형(타입) 확인 : int 7개, object 4개
## object 컬럼은 인코딩 필요
## 컬럼 수가 많지 않을 때는 info() 에서 결측치가 있는지 정도는 확인 가능 : Non-Null Count 가 모두 8799 로 결측치 X
print(X_train.info(), '\n')

## 기초 통계값 확인 : Cost_of_the_Product 와 Weight_in_gms 컬럼만 단위가 비교적 큼 -> 스케일링 작업을 통해 조정 가능
print(X_train.describe(), '\n')

## object 컬럼의 unique 개수 확인 : 작게는 2개부터 많게는 5개
print(X_train.describe(include='O'), '\n')

## test 데이터셋에 있는 object 컬럼도 확인 : test 데이터도 train 데이터와 유사한 형태
print(X_test.describe(include='O'), '\n')

## 결측치 여부 확인 : train 에는 결측치 없음
print(X_train.isnull().sum(), '\n')

## test 데이터의 결측치 확인
## sum() 하나 더 붙여서 결측치 수의 합 출력 : 0 출력시 결측치를 모두 더한 개수가 0이라는 의미
print(X_test.isnull().sum().sum(), '\n')

## Reached.on.Time_Y.N 컬럼의 종류에 따른 개수 각각 확인 : 0은 5236개, 1은 3563개
print(y_train['Reached.on.Time_Y.N'].value_counts(), '\n')

(8799, 11) (8799, 2)
      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   8440               B             Ship                    4   
1  10849               D             Ship                    3   
2   9478               B             Ship                    4   
3   2968               B             Ship                    6   
4   6964               B             Road                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  284                4                low   
1                2                  201                4             medium   
2                4                  181                3             medium   
3                2                  236                4                low   
4                1                  282                4                low   

  Gender  Discount_offered  Weight_in_gms  
0      F                 4           4795  
1      M           

In [4]:
# 4. 데이터 전처리
## 자료형이 object 인 컬럼 확인하고 레이블 인코딩 진행 -> 인코딩 전 후 결과 확인
## 인코딩 전의 데이터프레임을 보여주기 위해 display() 함수 활용
## display() : 데이터프레임을 보기 좋게 출력하지만 시험 환경에서는 지원X (print 사용)
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include='object').columns

display(X_train[cols].head())

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    
display(X_train[cols].head())

## ID 값은 데이터 수만큼 종류가 존재할 경우 큰 의미가 없다고 판단해 일반적으로는 삭제
## 삭제하지 않아도 머신러닝 모델이 중요도를 매우 낮게 적용하므로 괜찮지만, 숫자가 아닌 문자일 경우 별도 처리 필요
## ID 가 이커머스 데이터 같은(ex.제품모델ID) 유의미한 데이터(ex.반복되는 ID)라면 삭제 X
## train 데이터는 ID 컬럼 삭제, test 데이터의 ID 는 예측 csv 파일 생성시 필요 -> pop() 함수에 해당 컬럼 대입
X_train = X_train.drop('ID', axis=1)
X_test_id = X_test.pop('ID')

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
0,B,Ship,low,F
1,D,Ship,medium,M
2,B,Ship,medium,F
3,B,Ship,low,F
4,B,Road,low,F


,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
0,1,2,1,0
1,3,2,2,1
2,1,2,2,0
3,1,2,1,0
4,1,1,1,0


---
#### 💡 원-핫 인코딩 vs 레이블 인코딩
- 2개 다 해보고 검증 데이터를 활용한 머신러닝 평가가 좋은 인코딩 선택
---

In [5]:
# 5. 검증 데이터 분할
## train 데이터를 활용해 검증 데이터(20%) 분할
## random_state 는 특정 숫자로 고정해야 매번 동일한 분할을 하므로 필수
target = y_train.pop('Reached.on.Time_Y.N')

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size=0.2, random_state=0)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [6]:
# 6. 머신러닝 학습 및 평가
## 성능이 좋으면서 데이터에 민감하지 않은 모델 : 랜덤포레스트, 라이트지비엠(부스팅 계열)
## roc-auc 는 1에 가까울수록 성능이 좋은 모델
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

## 로지스틱 회귀 모델
lr = LogisticRegression(random_state=0)
lr.fit(X_tr, y_tr)
pred = lr.predict_proba(X_val)
print('로지스틱 회귀 :', roc_auc_score(y_val, pred[:, 1]))

## 의사결정 나무 모델
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr, y_tr)
pred = dt.predict_proba(X_val)
print('의사결정 나무 :', roc_auc_score(y_val, pred[:, 1]))

## 랜덤포레스트 모델
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)
print('랜덤포레스트 :', roc_auc_score(y_val, pred[:, 1]))

## Xgboost 모델
xg = xgb.XGBClassifier(random_state=0)
xg.fit(X_tr, y_tr)
pred = xg.predict_proba(X_val)
print('Xgboost :', roc_auc_score(y_val, pred[:, 1]))

## LightGBM 모델
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict_proba(X_val)
print('LightGBM :', roc_auc_score(y_val, pred[:, 1]))

c:\Users\babys\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


로지스틱 회귀 : 0.7294849094567405
의사결정 나무 : 0.648336686787391
랜덤포레스트 : 0.7605499664654595
Xgboost : 0.7570476190476191
LightGBM : 0.7657478202548625


In [7]:
# 7. 예측 및 결과 파일 생성
## test 데이터로 머신러닝 모델을 활용해 다시 한 번 예측
## train_test_split() 함수 사용 후 반드시 test 데이터로 다시 예측 실시
pred = lg.predict_proba(X_test)
print(pred)

## 문제에서 roc_auc 를 평가 기준으로 사 용한다면 예측방법인 predict, predict_proba 중 predict_proba 활용해 확률값을 예측할 것
## predict_proba 사용하면 이 데이터에서는 0일 확률과 1일 확률값이 존재하며 이 중에서 1일 확률값을 선택(pred[:, 1])
## 제출 양식 형태로 데이터프레임 생성 -> ID 값과 예측(확률)값 -> 컬럼명 : ID, Reached.on.Time_Y.N
submit = pd.DataFrame({'ID':X_test_id, 'Reached.on.Time_Y.N':pred[:, 1]})
print(submit.head())

submit.to_csv('result.csv', index=False)

pd.read_csv('result.csv')

[[0.59817028 0.40182972]
 [0.61564242 0.38435758]
 [0.703895   0.296105  ]
 ...
 [0.55070623 0.44929377]
 [0.52790166 0.47209834]
 [0.60239454 0.39760546]]
      ID  Reached.on.Time_Y.N
0   8285             0.401830
1  10192             0.384358
2   8675             0.296105
3   5753             0.431765
4    448             0.988722


,ID,Reached.on.Time_Y.N
0,8285,0.401830
1,10192,0.384358
2,8675,0.296105
3,5753,0.431765
4,448,0.988722
...,...,...
2195,7193,0.320605
2196,4445,0.401399
2197,7390,0.449294
2198,9977,0.472098


In [8]:
# 8-1. 성능 개선
## 베이스라인 모델 완성되면 첫 번째 '제출' 버튼 클릭
## 베이스라인 점수보다 높은 점수를 얻기 위해 데이터 전처리(인코딩, 스케일링) 진행

# 데이터 전처리 : 레이블 인코딩(0.7657478202548625)을 원-핫 인코딩(0.764689470154259으로 변경 -> 점수 떨어지므로 레이블 인코딩 그대로 사용
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv('./data/X_test.csv')
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

# 원-핫 인코딩
X_train = pd.get_dummies(X_train)
x_test = pd.get_dummies(X_test)

# train ID 삭제, test ID 다른 변수에 옮기기
X_train = X_train.drop('ID', axis=1)
X_test_id = X_test.pop('ID')

# 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=0)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 머신러닝 학습 및 평가
import lightgbm  as lgb
from sklearn.metrics import roc_auc_score
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict_proba(X_val)
print('roc_auc :', roc_auc_score(y_val, pred[:, 1]))

(7039, 19) (1760, 19) (7039,) (1760,)
roc_auc : 0.764689470154259


In [9]:
# 8-2. 성능 개선
# 기존 베이스라인 모델에서 Min-Max 스케일링 추가
## 스케일링을 적용할 컬럼을 선택 방법
### - 컬럼명을 직접 list 로 만드는 방법
### - X_train.select_dtypes(exclude='object').columns 코드 활용해 object 제외한 컬럼 선택하는 방법

# 데이터 전처리 : 스케일링 전(0.7657478202548625)보다 스케일링 후(0.7685566733735747) 가 성능 조금 더 향상
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv('./data/X_test.csv')
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    
# train ID 삭제, test ID 다른 변수에 옮기기
X_train = X_train.drop('ID', axis=1)
X_test_id = X_test.pop('ID')

# 스케일링
from sklearn.preprocessing import MinMaxScaler
cols = X_train.select_dtypes(exclude='object').columns
scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.transform(X_test[cols])

# 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=0)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)


# 머신러닝 학습 및 평가
import lightgbm as lgb
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict_proba(X_val)
print('roc_auc :', roc_auc_score(y_val, pred[:, 1]))

(7039, 10) (1760, 10) (7039,) (1760,)
roc_auc : 0.7685566733735747


In [10]:
# 9. 성능 개선 후 두 번째 csv 파일 만들고 '제출' 버튼 재클릭
pred = lg.predict_proba(X_test)
submit = pd.DataFrame({'ID': X_test_id, 'Reached.on.Time_Y.N': pred[:, 1]})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv').head())

      ID  Reached.on.Time_Y.N
0   8285             0.378522
1  10192             0.444619
2   8675             0.354102
3   5753             0.554405
4    448             0.986517
